# Coche autónomo mediante Deep Learning

Autor: Andrés Martínez Martínez

##Imports

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# common use
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# machine learning
import tensorflow as tf
import keras
import cv2

print( f'tf.version: {tf.__version__}')
print( f'keras.version: {keras.__version__}')

##Load data

In [ ]:
# import images and get a list with its paths
data_dir = '/content/drive/My Drive/Colab Notebooks/Autonomous Car/data2' # keras needs this path to create the dataset
file_list = os.listdir(data_dir + '/images')
file_list = sorted(file_list)

# get list of angles from file names, 95 degrees is follow straight

steering_angles = [int(name[-7:-4]) for name in file_list]

print('number of images: %d' % len(file_list))

In [ ]:
# check distribution of steering angles
fig, axes = plt.subplots(figsize=(12,6))
plt.hist(steering_angles, bins=30, width=1)
plt.title('steering angles')
plt.show()

# Prepare data

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir, 
    labels=steering_angles, 
    label_mode='int', 
    color_mode='rgb',
    batch_size=32,
    image_size=(240,320),
    seed=123,
    validation_split=0.2,
    subset='training')

test_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir, 
    labels=steering_angles, 
    label_mode='int', 
    color_mode='rgb',
    batch_size=32,
    image_size=(240,320),
    seed=123,
    validation_split=0.2,
    subset='validation') 

# Visualize data

In [ ]:
images = []
while len(images) < 4:
  for images, labels in train_dataset.take(1):
    im_shape = images[1].shape
    for i in range(4):
      ax = plt.subplot(2, 2, i + 1)
      plt.imshow(images[i].numpy().astype('uint8'))
      plt.title(labels[i].numpy())
      plt.axis('off')
print(images[1].shape)

# Regression head

In [ ]:
model = tf.keras.Sequential([
  # preprocessing
  tf.keras.layers.Cropping2D(cropping=((80,0),(0,0))),
  tf.keras.layers.Resizing(61,157),
  tf.keras.layers.RandomRotation(0.01),
  tf.keras.layers.Rescaling(1./255),
  
  # conv layers
  keras.layers.Conv2D(24, 5, strides=2, activation='elu'),
  keras.layers.Conv2D(36, 5, strides=2, activation='elu'),
  keras.layers.Conv2D(48, 5, strides=2, activation='elu'),
  keras.layers.Conv2D(64, 3, activation='elu'), 
  keras.layers.Conv2D(64, 3, activation='elu'), 

  # fully connected layers
  tf.keras.layers.Flatten(),

  tf.keras.layers.Dense(100, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(50, activation='elu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  #tf.keras.layers.GlobalAveragePooling2D(),
  
  # output layer
  keras.layers.Dense(1),
])

# Configure model

In [ ]:
model.compile(loss='mse', 
              optimizer='adam')

In [ ]:
model.build((32,240,320,3))
model.summary()

# Training

In [ ]:
model_output_dir = '/content/drive/My Drive/Colab Notebooks/Autonomous Car/model'
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_output_dir,'autonomous_car_v4_check.h5'), verbose=1, save_best_only=True)

history = model.fit(train_dataset,
                    epochs=10,
                    verbose=1,
                    callbacks=[checkpoint_callback],
                    validation_data=test_dataset)

Epoch 1/10
98/98 [==============================] - ETA: 0s - loss: 1781.8411
Epoch 1: val_loss improved from inf to 885.73022, saving model to /content/drive/My Drive/Colab Notebooks/Autonomous Car/model/autonomous_car_v4_check.h5
98/98 [==============================] - 33s 317ms/step - loss: 1781.8411 - val_loss: 885.7302
Epoch 2/10
98/98 [==============================] - ETA: 0s - loss: 970.7657
Epoch 2: val_loss did not improve from 885.73022
98/98 [==============================] - 30s 300ms/step - loss: 970.7657 - val_loss: 900.5845
Epoch 3/10
98/98 [==============================] - ETA: 0s - loss: 953.8186
Epoch 3: val_loss improved from 885.73022 to 873.84412, saving model to /content/drive/My Drive/Colab Notebooks/Autonomous Car/model/autonomous_car_v4_check.h5
98/98 [==============================] - 31s 308ms/step - loss: 953.8186 - val_loss: 873.8441
Epoch 4/10
98/98 [==============================] - ETA: 0s - loss: 984.5508
Epoch 4: val_loss did not improve from 873.84

## Save model

In [ ]:
model.save(os.path.join(model_output_dir,'autonomous_car_v4.h5'))

# Print accuracy

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
ax.plot(loss[:10], label='Training Loss')
ax.plot(val_loss[:10], label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Mean Squared Error')
plt.title('MobileNet V2')
plt.xlabel('epoch',labelpad=2)
ax.yaxis.tick_right()
plt.show()